In [ ]:
!pip3 install pytorch_lightning 

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms, datasets
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning import Trainer

# Get device

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

# MNIST dataset

In [4]:
transform = transforms.Compose([
                transforms.ToTensor()
            ])

In [ ]:
train_val = torchvision.datasets.MNIST(root="data", train=True, download=True, transform=transform)
test = torchvision.datasets.MNIST(root="data", train=False, download=True, transform=transform)

# Split train - val

In [6]:
n_train = int(len(train_val) * 0.8)
n_val = len(train_val) - n_train

torch.manual_seed(0)
train, val = torch.utils.data.random_split(train_val, [n_train, n_val])

# Data Loader

In [7]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

# Define model - Lightning

In [39]:
class LearningModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.conv = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=(3,3))
    self.fc = nn.Linear(507, 10)
    self.feature_extractor = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2),
        nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1, padding=1),
        nn.ReLU()
    )
    
    self.classifier = nn.Sequential(
        nn.Linear(in_features=1080, out_features=540),
        nn.ReLU(),
        nn.Linear(in_features=540, out_features=10)
    )

    self.train_acc = pl.metrics.Accuracy()
    self.val_acc = pl.metrics.Accuracy()
    self.test_acc = pl.metrics.Accuracy()

  def forward(self, x):
    x = self.feature_extractor(x)
    x = x.view(x.size(0), -1)
    x = self.classifier(x)

    return x

  def training_step(self, batch, batch_idx):
    x, t = batch 
    y = self(x)
    
    loss = F.cross_entropy(y, t)
    self.log('train_loss: ', loss, on_step=False, on_epoch=True)
    self.log('train_acc: ', self.train_acc(y, t), on_step=False, on_epoch=True)

    return loss
  
  def validation_step(self, batch, batch_idx):
    x, t = batch
    y = self(x)

    loss = F.cross_entropy(y, t)
    self.log('val_loss: ', loss, on_step=False, on_epoch=True)
    self.log('val_acc: ', self.val_acc(y, t), on_step=False, on_epoch=True)

    return loss 
  
  def test_step(self, batch, batch_idx):
    x, t = batch 
    y = self(x)

    loss = F.cross_entropy(y, t)
    self.log('test_loss: ', loss, on_step=False, on_epoch=True)
    self.log('test_acc: ', self.test_acc(y, t), on_step=False, on_epoch=True)
  
  def configure_optimizers(self):
    optimizer = torch.optim.SGD(self.parameters(), lr=0.001, momentum=0.9)
    return optimizer

In [40]:
pl.seed_everything(0)
model = LearningModel()

trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, train_loader, val_loader)

result = trainer.test(test_dataloaders=test_loader)

Global seed set to 0
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name              | Type       | Params
-------------------------------------------------
0 | conv              | Conv2d     | 30    
1 | fc                | Linear     | 5.1 K 
2 | feature_extractor | Sequential | 50.7 K
3 | classifier        | Sequential | 589 K 
4 | train_acc         | Accuracy   | 0     
5 | val_acc           | Accuracy   | 0     
6 | test_acc          | Accuracy   | 0     
-------------------------------------------------
644 K     Trainable params
0         Non-trainable params
644 K     Total params
2.580     Total estimated model params size (MB)


Global seed set to 0



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc: ': 0.9868000149726868, 'test_loss: ': 0.040737126022577286}
--------------------------------------------------------------------------------
